In [1]:
import numpy as np
import pandas as pd
import random

# GA Class

In [14]:
class MancalaGA:
    
    def __init__(self, self_state, opponent_state, gen, popsize, generation, pc, pm):
        self.num_of_gen = gen
        self.pop_size = popsize
        self.num_of_generation = generation
        self.pc = pc
        self.pm = pm
        self.self_state = self_state # list state ai (lubang) *default [7,7,7,7,7,7,7]
        self.opponent_state = opponent_state # list state player (lubang) *default [7,7,7,7,7,7,7]
        
    
    def individu(self):
        return np.array([np.random.uniform(0,100) for i in range(self.num_of_gen)])
    
    def population(self):
        return np.array([self.individu() for i in range(self.pop_size)])
    
    def debug_lubang_representation(self):
        global h1,h2,h3,h4,h5
        populasi = self.population()
        
        
        for idx, lubang in enumerate(self.self_state):
            idx_lubang = idx
            isi_lubang = lubang

            # get value tiap heuristik
            h1 = self.h1(idx_lubang, isi_lubang)
            h2 = self.h2(idx_lubang, isi_lubang)
            h3 = self.h3(idx_lubang, isi_lubang)
            h4 = self.h4(idx_lubang, isi_lubang)
            h5 = self.h5(idx_lubang, isi_lubang)


            print(f"ini lubang ke {idx_lubang} dengan isi isi_lubang {isi_lubang}")
            print(f"nilai h1 adalah {h1}")
            print(f"nilai h2 adalah {h2}")
            print(f"nilai h3 adalah {h3}")
            print(f"nilai h4 adalah {h4}")
            print(f"nilai h5 adalah {h5}")
                
                

            print("\n")
                
    
    def debug(self):
        pass
            
            
    def calc_fitnes(self, individu, heuristic):
        wh = heuristic*individu
        return np.sum(wh[:self.num_of_gen-1]) - wh[-1]
    
    def self_scan(self):
        data = {}
        for idx, lubang in enumerate(self.self_state):
            h1 = self.h1(idx, lubang)
            h2 = self.h2(idx, lubang)
            h3 = self.h3(idx, lubang)
            h4 = self.h4(idx, lubang)
            h5 = self.h5(idx, lubang)
            
            data[idx] = np.array([h1,h2,h3,h4,h5])
        return data
    
    def candidate(self):
        pass
        
    
    def h1(self, idx_lubang, isi_lubang):
        '''
        Timbun sebanyak mungkin rock dalam satu lubang. 
        Karena pada akhir permainan, rock pada lubang akan dipindahkan semuanya pada lumbung. Lubang kanan semakin aman.
        '''
        # [0,1,2,3,4,5,6] [lumbung]
        if len(self.self_state)-(idx_lubang+isi_lubang) == 0:
            return 1.0
        return 0.0
    
    def h2(self, idx_lubang, isi_lubang):
        '''
        Pertahankan agar rock pada sisi kita sebanyak mungkin. (versi umum H1)
        '''
        if isi_lubang == 0:
            return 0.0
        
        total_batu_before = np.sum(self.self_state)
        
        # simulasikan perpindahan batu pada lubang
        cp_state = self.self_state[:]
        
            
        # pecah list menjadi kiri dan kanan
        kiri = cp_state[:idx_lubang]
        tengah = [0]
        kanan = cp_state[idx_lubang+1:]
        
        jml_batu = isi_lubang
        new_kanan = []
        for i in kanan:
            
            if jml_batu == 0:
                break
            else:
                new_kanan.append(i+1)
            
            jml_batu -= 1
        
        new_state = kiri+tengah+kanan

        
        # jumlahkan setiap batu untuk kondisi terbaru
        total_batu_after = np.sum(new_state)
        
        return total_batu_after/total_batu_before
    
    def h3(self, idx_lubang, isi_lubang):
        '''
        Mempertahankan gerakan memindah rock sebanyak mungkin
        '''
        # [0,1,2,3,4,5,6] [lumbung]
        if len(self.self_state) == (idx_lubang+isi_lubang):
            return 1.0
        return 0.0
    
    def h4(self, idx_lubang, isi_lubang):
        '''
        Memaksimalkan jumlah rock pada lumbung. Dengan memilih langkah mencuri
        '''
        target = idx_lubang + isi_lubang
        
        if isi_lubang == 0:
            return 0.0
        elif 0 not in self.self_state:
            return 0.0
        elif target >= len(self.self_state):
            # out of range
            return 0.0
        elif self.self_state[target] == 0:
            # jika target nilainya 0
            # curi
            jml_curi = self.opponent_state[target]
            return 1-(1/jml_curi)
        else:
            return 0.0
#         if isi_lubang == 0:
#             return 0.0
        
#         if 0 not in self.self_state:
#             # jika ga ada 0 dalam lubang kita, berarti ga ada bisa yang dicuri
#             return 0.0
        
#         # jumlah perpindahan batu
#         jml_pindah = idx_lubang+isi_lubang
#         # print('jumlah pindah ', jml_pindah)
        
#         # jika kelebihan langkah batu
#         if len(self.self_state) < jml_pindah:
#             # out of range
#             return 0.0
        
#         if self.self_state[jml_pindah] == 0 and jml_pindah <= len(self.self_state):
#             # lakukan pencurian
#             jml_curi = self.opponent_state[jml_pindah]
#             return 1-(1/jml_curi)
#         else:
#             # ga ada yang bisa dicuri
#             return 0.0

        
    def h4_new(self, idx_lubang, isi_lubang):
        
        target = idx_lubang + isi_lubang
        
        if isi_lubang == 0:
            return 0.0
        elif 0 not in self.self_state:
            return 0.0
        elif target > len(self.self_state):
            # out of range
            return 0.0
        elif self.self_state[target] == 0:
            # jika target nilainya 0
            # curi
            jml_curi = self.opponent_state[jml_pindah]
            return 1-(1/jml_curi)
        else:
            return 0.0
        
    
    def h5(self, idx_lubang, isi_lubang):
        '''
        h5 pada excel dihapus diganti dengan h6 diexcel
        yaitu strategi bertahan
        Menjaga score musuh seminimal mungkin. (heuristik dengan mempertimbangkan 2 gerakan musuh kedepan.
        '''
        # [0,1,2,3,4,5,6] [lumbung]
        if isi_lubang == 0:
            return 0.0
        
        jml_batu_ke_musuh = abs(len(self.self_state) - (idx_lubang+isi_lubang))
        if jml_batu_ke_musuh > 0: # berarti ada batu yang masuk ke lubang musuh
            jml_batu_ke_musuh = len(self.self_state) if jml_batu_ke_musuh > len(self.self_state) else jml_batu_ke_musuh
            return jml_batu_ke_musuh/len(self.self_state)
        else:
            return 0.0
    
    def turnamen(individu1, individu2):
        pass
    
    def selection(self, population, verbose=False):
        '''
        menggunakan metode turnamen dari individu yang berada dalam kandidat
        '''
        scan = self.self_scan()
        
        self.kandidat = []
        
        self.rata_fitness = [0,0,0,0,0,0,0,0,0,0]
        for lubang, heuristic in scan.items():
            if verbose:
                print(f"===========================lubang {lubang}======================================")
                print("\n")
                temp = []
                for idx, individu in enumerate(population):
                    data_individu = {
                        'lubang': lubang,
                        'heuristic': heuristic,
                        'individu': {
                            'individuke':idx,
                            'gen':individu
                        },
                        'fitness': self.calc_fitnes(individu, heuristic)
                    }
                    
                    
                    temp.append(data_individu)
                    self.rata_fitness[idx] += data_individu['fitness']
                    print(data_individu)
                    print("\n")
                

                best_individu_each_lubang = sorted(temp, key=lambda p : p.get('fitness'), reverse=True)
                print(f'best individu di lubang {lubang} adalah {best_individu_each_lubang[0]}')
                # append to kandidat
                self.kandidat.append(best_individu_each_lubang[0])
                print(f"================================================================================")
                print("\n")
                
                
                # masukan individu terbaik dari setiap lubang kedalam list kandidat
                
            else:
                pass
            
    def turnamen(self, pop):
        kandidat= []
        result = self.rata_fitness
        while ((len(kandidat)<4)):
            x = random.randint(0,9)
            if(x not in kandidat):
                kandidat.append(x)
            else:
                pass
#         print("daftar kandidat yang akan di turnamentkan ", kandidat)
#         #kandidat[0] vs kandidat[1] , kandidat[2] vs kandidat[3]
#         print(pop[kandidat[0]]," Dengan Fitness ",result[kandidat[0]])
#         print(pop[kandidat[1]]," Dengan Fitness ",result[kandidat[1]])
#         print("=============================================================")
#         print(pop[kandidat[2]]," Dengan Fitness ",result[kandidat[2]])
#         print(pop[kandidat[3]]," Dengan Fitness ",result[kandidat[3]])
#         print("=======================TOURNAMENT DIMULAI=============================")
        if(result[kandidat[0]] > result[kandidat[1]]):
            x = pop[kandidat[0]]
        else:
            x = pop[kandidat[1]]

        if(result[kandidat[2]] > result[kandidat[3]]):
            y = pop[kandidat[2]]
        else:
            y = pop[kandidat[3]]
#         print("========================================================================")
#         print("x = ",x)
#         print("y = ",y)
        return x, y
    
    def crossover(self, individu1, individu2):
        '''
        crossover dengan single point mutation
        
        return new offspring
        '''
        prandom = np.random.uniform(0,1)
        
        if prandom < self.pc:
            # lakukan crossover
            point = random.randint(0,3)
        
            papa = list(individu1[:point])
            
            
            mama = list(individu2[point:])
    
            
            offspring = papa+mama 


            
#             return offspring
            return np.array(offspring)     
        else:
            print("masuk else")
            '''
            return kan random mama atau papa
            '''
            prob=random.randint(0,1)
            if(prob==0):
                offspr = list(individu1)
                return np.array(offspr)
            else:
                offspr = list(individu2)
                return np.array(offspr)
    
    def mutation(self, individu, num_of_gen):
        scan = self.self_scan()
        '''
        mutasi
        lakukan mutasi hanya untuk 2 gen dalam 1 individu jika propbabiliti individu lebih kecil dari pm
        '''
        
        pidv = np.random.uniform(0,1)
        print(pidv)
        if pidv < self.pm:
            
            # lakukan pemilihan random gen
            gen_bermutasi = np.random.choice([i for i in range(self.num_of_gen)], num_of_gen)
            #print(gen_bermutasi)
            
            for i in gen_bermutasi:
                
                individu[i] = np.random.uniform(0,100)
            #hitung FV langsung
            heur=ga.self_scan()
            temporary=[]
            for i in range(len(heur)):
                hasil=heur[i]*individu
                temporary.append(hasil)
            res=0
            for j in range(len(temporary)):
                res+= temporary[j][0]+temporary[j][1]+temporary[j][2]+temporary[j][3]-temporary[j][4]
            result=res
            return individu,result
        
        else:
            heur=ga.self_scan()
            temporary=[]
            for i in range(len(heur)):
                hasil=heur[i]*individu
                temporary.append(hasil)
            res=0
            for j in range(len(temporary)):
                res+= temporary[j][0]+temporary[j][1]+temporary[j][2]+temporary[j][3]-temporary[j][4]
            result=res
            return individu,result
        
    def elitism(self,daftar,new_individu,fv_new):
        temp=daftar[0]
        hapus_idx=0
        for i in range(len(daftar)):
            if(daftar[i]<temp):
                temp=daftar[i]
                hapus_idx = i
                print("hapus idx=",hapus_idx)
        if(fv_new>=daftar[hapus_idx]):
            print('populasi yang terganti =',hapus_idx)
            daftar[hapus_idx]=fv_new
            pop[hapus_idx]=new_individu
            return pop,daftar
        else:
            print("generasi ini tidak terganti")
            return pop,daftar
    def moving(self,daftar):
        temp=daftar[0]
        idx_terpilih=0
        for i in range(len(daftar)):
            if(daftar[i]>temp):
                temp=daftar[i]
                idx_terpilih = i
                print("individu terpilih adalah individu ke ",idx_terpilih)
        return idx_terpilih

## RUN

In [15]:
'''
setting param
'''


SELF_STATE = [1,0,2,3,0,5,0]
OPPONENT_STATE = [6,7,8,9,2,4,12]
NUMBER_OF_GEN = 5
NUMBER_OF_POPULATION = 10
NUMBER_OF_GENERATION = 10
PC = 0.8
PM = 0.5


ga = MancalaGA(SELF_STATE, OPPONENT_STATE, NUMBER_OF_GEN, NUMBER_OF_POPULATION, NUMBER_OF_GENERATION, PC, PM)

In [16]:
ga.self_scan()

{0: array([0.        , 0.90909091, 0.        , 0.85714286, 0.85714286]),
 1: array([0., 0., 0., 0., 0.]),
 2: array([0.        , 0.81818182, 0.        , 0.5       , 0.42857143]),
 3: array([0.        , 0.72727273, 0.        , 0.91666667, 0.14285714]),
 4: array([0., 0., 0., 0., 0.]),
 5: array([0.        , 0.54545455, 0.        , 0.        , 0.42857143]),
 6: array([0., 0., 0., 0., 0.])}

In [17]:
pop = ga.population()
pop

array([[ 3.53787505, 61.68520212, 79.24112718, 49.16370169,  8.51541387],
       [47.70792723, 74.17930502, 20.55259016, 31.23437081, 52.63487357],
       [97.4096484 , 23.75308813, 19.11436495, 97.07731255,  6.03832801],
       [36.89891558,  9.49840088, 12.89235428, 95.48545623,  7.87440338],
       [60.73403587, 91.03382233, 95.45819328, 58.46949919, 90.54048554],
       [ 7.84416269, 56.05857318, 39.53533511, 93.55501822, 56.00981895],
       [ 3.16169406, 51.43383018, 28.21242627, 72.13997936, 63.45871042],
       [91.92324711,  0.17543134, 91.43083875,  0.76997823,  4.54749264],
       [81.4445194 , 44.59991683, 53.04101399, 35.32183829, 59.64972251],
       [80.43361753, 68.16796579, 19.12780784, 36.17996375,  0.32733229]])

In [18]:
for iterasi in range(0,10):
    print("###################### GENERASI KE ",iterasi," #######################")
    ga.selection(pop, verbose=True)
    ga.rata_fitness
    turnamen = ga.turnamen(pop)
    c = ga.crossover(turnamen[0], turnamen[1])
    new_individu,fv_new=ga.mutation(c,2)
    daftar=ga.rata_fitness
    pop,f=ga.elitism(daftar,new_individu,fv_new)

###################### GENERASI KE  0  #######################
===========================lubang 0======================================


{'lubang': 0, 'heuristic': array([0.        , 0.90909091, 0.        , 0.85714286, 0.85714286]), 'individu': {'individuke': 0, 'gen': array([ 3.53787505, 61.68520212, 79.24112718, 49.16370169,  8.51541387])}, 'fitness': 90.91884602767519}


{'lubang': 0, 'heuristic': array([0.        , 0.90909091, 0.        , 0.85714286, 0.85714286]), 'individu': {'individuke': 1, 'gen': array([47.70792723, 74.17930502, 20.55259016, 31.23437081, 52.63487357])}, 'fitness': 49.09244375449804}


{'lubang': 0, 'heuristic': array([0.        , 0.90909091, 0.        , 0.85714286, 0.85714286]), 'individu': {'individuke': 2, 'gen': array([97.4096484 , 23.75308813, 19.11436495, 97.07731255,  6.03832801])}, 'fitness': 99.62713180933825}


{'lubang': 0, 'heuristic': array([0.        , 0.90909091, 0.        , 0.85714286, 0.85714286]), 'individu': {'individuke': 3, 'gen': array([3

hapus idx= 4
populasi yang terganti = 4
###################### GENERASI KE  8  #######################
===========================lubang 0======================================


{'lubang': 0, 'heuristic': array([0.        , 0.90909091, 0.        , 0.85714286, 0.85714286]), 'individu': {'individuke': 0, 'gen': array([ 3.53787505, 61.68520212, 79.24112718, 49.16370169,  8.51541387])}, 'fitness': 90.91884602767519}


{'lubang': 0, 'heuristic': array([0.        , 0.90909091, 0.        , 0.85714286, 0.85714286]), 'individu': {'individuke': 1, 'gen': array([97.4096484 , 23.75308813, 19.11436495, 97.07731255,  6.03832801])}, 'fitness': 99.62713180933825}


{'lubang': 0, 'heuristic': array([0.        , 0.90909091, 0.        , 0.85714286, 0.85714286]), 'individu': {'individuke': 2, 'gen': array([97.4096484 , 23.75308813, 19.11436495, 97.07731255,  6.03832801])}, 'fitness': 99.62713180933825}


{'lubang': 0, 'heuristic': array([0.        , 0.90909091, 0.        , 0.85714286, 0.85714286]), 'indi

In [23]:
pop
ga.rata_fitness
ga.moving(daftar)

individu terpilih adalah individu ke  5


5

In [ ]:
pop


In [ ]:
f

In [ ]:
temp=daftar[0]
for i in range(len(daftar)):
    if(daftar[i]<temp):
        temp=daftar[i]
        hapus_idx=i
    else:
        pass
print('temp =',hapus_idx)
daftar[hapus_idx]=fv_new
pop[hapus_idx]=new_individu

In [ ]:
daftar

In [ ]:
pop

In [ ]:
heur=ga.self_scan()
temporary=[]
for i in range(len(heur)):
    hasil=heur[i]*new_individu
    temporary.append(hasil)
res=0
for j in range(len(temporary)):
    res+= temporary[j][0]+temporary[j][1]+temporary[j][2]+temporary[j][3]-temporary[j][4]
result=res/5
print(result)